In [28]:
#Run this code to make the Directory's
# import os
#
# curDir = os.getcwd()
# os.mkdir(os.getcwd() + "/images")
# os.mkdir(os.getcwd() + "/models")
# os.chdir(os.getcwd() + "/images")
# os.mkdir(os.getcwd() + "/all_images")
# os.mkdir(os.getcwd() + "/detected_persons")
# os.mkdir(os.getcwd() + "/not_relevant_label")
# os.chdir(curDir)


### Save the Directory's Pathes

In [10]:
import yaml
import os
with open("config.yaml" , "r") as config_file:
    config = yaml.load(config_file , Loader=yaml.FullLoader)

PATH_TO_DETECTED_PERSONS = config['PATH_TO_DETECTED_PERSON']
PATH_TO_ALL_IMAGES = config['PATH_TO_ALL_IMAGES']
PATH_TO_MODEL_DIR = config['PATH_TO_MODEL']
PATH_TO_NOT_RELEVANT_LABEL = config['PATH_TO_NOT_RELEVANT_LABEL']

In [11]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf

tf.get_logger().setLevel('ERROR')  # Suppress TensorFlow logging (2)

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [12]:
def get_Image_Paths_Array(path):
    """
    @path ist ein pfad zu einem Ordner mit unterordner die Bilder Inhalten
    :returns ein Array mit den pfaden zu jedem Bild in den Unterordner
    """
    # assign directory
    #path = '/Volumes/Anas SSD/Bachelorarbeit/Bilder(Sammlung1)/'
    array = []
    for image in os.listdir(path):
        fullFileName = os.path.join(path, image)
        array.append(fullFileName)
    return array


IMAGE_PATHS = get_Image_Paths_Array(PATH_TO_ALL_IMAGES)
len(IMAGE_PATHS)

9122

In [13]:
import numpy as np
def clear_processed_images(image_array):
    processed_images = os.listdir(PATH_TO_DETECTED_PERSONS) + os.listdir(PATH_TO_NOT_RELEVANT_LABEL)
    for image in processed_images:
        image_name = os.path.join(os.getcwd() + "/images/all_images/", image)
        index = np.argwhere(image_array == image_name)
        image_array = np.delete(image_array, index)
    return image_array


IMAGE_PATHS = clear_processed_images(IMAGE_PATHS)

In [14]:
# Download labels file
def download_labels(filename):
    base_url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/'
    label_dir = tf.keras.utils.get_file(fname=filename,
                                        origin=base_url + filename,
                                        untar=False)
    label_dir = pathlib.Path(label_dir)
    return str(label_dir)

LABEL_FILENAME = 'mscoco_label_map.pbtxt'
PATH_TO_LABELS = download_labels(LABEL_FILENAME)

print("Pfad: " + PATH_TO_LABELS)

Pfad: /Users/anasarodake/.keras/datasets/mscoco_label_map.pbtxt


In [15]:
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

# Es ist notwendig das matplotlib Backend nach dem viz_utils import wieder umzustellen damit die inline plot funktion funktioniert, da in visualization_utils das matplotlib Backend überschrieben wird

#matplotlib.use("module://matplotlib_inline.backend_inline")
#print(matplotlib.get_backend())

PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR
print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...Done! Took 26.387078046798706 seconds


In [16]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [17]:
#% matplotlib inline
import matplotlib

matplotlib.use("module://matplotlib_inline.backend_inline")
#matplotlib.use("MacOSX")

from PIL import Image
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')  # Suppress Matplotlib warnings


def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))



In [18]:

#detected_Images = np.unique(detected_Images)
for image_path in IMAGE_PATHS:
    print('Running inference for {}... '.format(image_path), end='')

    if '.DS_Store' in image_path:
        continue

    image_np = load_image_into_numpy_array(image_path)
    if len(image_np.shape) == 2:
        image_np = np.stack((image_np,) * 3, axis=-1)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    time1 = time.time()
    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    ################################################################################################
    """MOVE THE IMAGES WITH PERSONS INTO ANOTER DIR"""
    indexOfEveryPredictedOne = np.where(detections['detection_classes'] == 1)[
        0]  #save the indexes of every value =1 in the array "detection_classes"
    scoreOfAllOne = []
    for index in indexOfEveryPredictedOne:
        score = detections['detection_scores'][index]
        scoreOfAllOne.append(score)  # Save the scores of everey "class 1"  has been predicted

    if any(num > 0.3 for num in scoreOfAllOne):
        destinationPath = PATH_TO_DETECTED_PERSONS
        os.system("cp " + "\"" + image_path + "\"" + " \"" + destinationPath + "\"")
    else:
        destinationPath = PATH_TO_NOT_RELEVANT_LABEL
        os.system("cp " + "\"" + image_path + "\"" + " \"" + destinationPath + "\"")

    ################################################################################################
    time2 = time.time()
    print("took: ", time2 - time1)
    image_np_with_detections = image_np.copy()

    # viz_utils.visualize_boxes_and_labels_on_image_array(
    #     image_np_with_detections,
    #     detections['detection_boxes'],
    #
    #     detections['detection_classes'],
    #     detections['detection_scores'],
    #     category_index,
    #     use_normalized_coordinates=True,
    #     max_boxes_to_draw=200,
    #     min_score_thresh=.3,
    #     agnostic_mode=False)
    #
    # plt.figure()
    # plt.imshow(image_np_with_detections)
    # print('Done')
    # plt.show()

end_time = time.time()
print("###################################################################################################")
print("completed in", end_time - start_time)
# sphinx_gallery_thumbnail_number = 2

Running inference for ./images/all_images/BA 2005-01201 b_Scan01.jpg... took:  4.361147880554199
Running inference for ./images/all_images/BA 2005-00563_Scan01.jpg... took:  0.7257556915283203
Running inference for ./images/all_images/BA 2004-12965-0584_Scan01.jpg... took:  0.5111451148986816
Running inference for ./images/all_images/BA 2004-00200_Dok01.jpg... took:  0.47658205032348633
Running inference for ./images/all_images/BA 2004-02267_Scan01.jpg... took:  0.5593893527984619
Running inference for ./images/all_images/BA 2004-13069-15_Scan01.jpg... took:  0.5771162509918213
Running inference for ./images/all_images/BA 2004-06064_Scan01.jpg... took:  0.6369140148162842
Running inference for ./images/all_images/BA 2005-01260_Scan01.jpg... took:  0.6199800968170166
Running inference for ./images/all_images/BA 2004-13061-41_Scan01.jpg... took:  0.5590131282806396
Running inference for ./images/all_images/BA 2004-00036_Dok01.jpg... took:  0.6867098808288574
Running inference for ./image

KeyboardInterrupt: 